In [1]:
!pip install cvzone
!pip install mediapipe

In [7]:
import cv2
import cvzone
from cvzone.PoseModule import PoseDetector
import math
import numpy as np

counter = 0
direction = 0

cap = cv2.VideoCapture('vid1.mp4')
pd = PoseDetector(trackCon=0.70, detectionCon=0.70)

# Get the frames per second (fps) and total number of frames in the video
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (1000, 500))

def angles(lmlist, p1, p2, p3, p4, p5, p6, drawpoints):
    global counter
    global direction

    if len(lmlist) >= max(p1, p2, p3, p4, p5, p6):
        point1 = lmlist[p1]
        point2 = lmlist[p2]
        point3 = lmlist[p3]
        point4 = lmlist[p4]
        point5 = lmlist[p5]
        point6 = lmlist[p6]

        if len(point1) >= 2 and len(point2) >= 2 and len(point3) >= 2 and len(point4) >= 2 and len(point5) >= 2 and len(point6) >= 2:
            x1, y1 = point1[:2]
            x2, y2 = point2[:2]
            x3, y3 = point3[:2]
            x4, y4 = point4[:2]
            x5, y5 = point5[:2]
            x6, y6 = point6[:2]

            if drawpoints:
                cv2.circle(img, (int(x1), int(y1)), 10, (255, 0, 255), 5)
                # Draw other points and lines as needed...

            # Calculate angles
            lefthandangle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
            righthandangle = math.degrees(math.atan2(y6 - y5, x6 - x5) - math.atan2(y4 - y5, x4 - x5))

            leftHandAngle = int(np.interp(lefthandangle, [-30, 180], [100, 0]))
            rightHandAngle = int(np.interp(righthandangle, [34, 173], [100, 0]))

            left, right = leftHandAngle, rightHandAngle

            if left >= 70 and right >= 70:
                if direction == 0:
                    counter += 1
                    direction = 1
            if left <= 70 and right <= 70:
                if direction == 1:
                    direction = 0

            # Display the push-up count
            cv2.rectangle(img, (0, 0), (120, 120), (255, 0, 0), -1)
            cv2.putText(img, str(int(counter)), (20, 70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1.6, (0, 0, 255), 7)

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break  # Break the loop when the video ends

    img = cv2.resize(img, (1000, 500))
    cvzone.putTextRect(img, 'AI Push Up Counter', [345, 30], thickness=2, border=2, scale=2.5)
    pd.findPose(img, draw=0)
    lmlist, bbox = pd.findPosition(img, draw=0, bboxWithHands=0)

    angles(lmlist, 11, 13, 15, 12, 14, 16, drawpoints=1)

    # Write the frame to the output video
    out.write(img)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()
